<a href="https://colab.research.google.com/github/mehrerm/TFM/blob/main/notebooks/01_load_and_clean_DIRAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación de modelos de puntuación del Riesgo de Impago

En esta aplicación implementaremos un modelo de puntuación de riesgo poblacional a la mortalidad debido a los cánceres más comunes y su relación con las cantidades y tecnologías usadas en radioterapia.


riesgo de impago siguiendo la metodología  que hemos analizado en la presentación de clase (selección de variables mediante valor de la información, tramificación de variables continuas, agrupación de categorías, transformación woe de variables, estimación de modelos de regresión logística, ....)

Existen diferentes librerías que incorporan funciones con los diferentes procedimientos ya programados que nos facilitan mucho la tarea. Una de estas librerías es `scorecardpy` [librería scorecardpy](https://pypi.org/project/scorecardpy/) que estima tarjetas de puntuación *lineales* utilizando regresiones logísticas. Esta librería nació inicialmente en R, y lamentablemente la versión de Python da algunos errores de adaptación a las últimas versiones de Pandas. Su desarrollador remite a utilizar la versión estable de R (librería en R 'scorecard').

Así que en su lugar de esta librería utilizaremos la librería `optBinning` [librería OptBinning](http://gnpalencia.org/optbinning/) que en realidad recoge (y en mi opinión mejora) la principal función de la librería `scorecardpy`  

## Creación de un entorno e instalación de librerías
# como en este caso usaremos Colab, no necesitaremos instalar librerías, pero sí cargarlas en este entorno. Se procuarará importar


In [ ]:
#Cargo o importo pandas, numpy, Matplotlib,
from google.colab import files

uploaded = files.upload()





import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para el análisis descriptivo inicial de contraste de asociación importo el test Chi2 y el anova
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway


# Librería para hacer la tramificación, agrupación y transformación WOE
from optbinning import Scorecard, BinningProcess, OptimalBinning
from optbinning.scorecard import plot_auc_roc, plot_cap, plot_ks, ScorecardMonitoring

# Scikit-learn para dividir la muestra y para estimar el modelo de regresión logística (sólo si no se quiere utilizar
# la función optbinning.scorecard que ya lo incropora)
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

# Carga, Exploración y Preparación de los datos sobre equipos de radioterapia y pacientes que utilizaremos
## Carga de los datos
Se utilizará una base de datos de créditos de la IAEA junto con los de la OMS, son datos reales, se buscará primeramente un estudio de un solo año (2022) ya que se busca algo lo más actual, completo y que tenga lo menos posible la influencia del COVID-19.

 Eso significa que las magnitudes de cantidades (expresadas en Marcos Alemanes), sean de difícil interpretación para el día de hoy. Sin embargo el signo y el sentido de las variables utilizadas para predecir el riesgo de impago de los futuros clientes permanece todavía de plena utilidad.

Los datos pueden descargarse en la página de la IAEA en el apartado de DIRAC https://dirac.iaea.org/Query/Countries  donde aparecen reflejados los datos de los equipos de radioterapia y el año de sus ultimas actualizaciones q nivel the hardware, por otro lado, se descargaron los datos de la OMS, en el Global Cancer Observatory, donde se descargaron tanto las incidencias como las mortalidades por cáncer a nivel mundial https://gco.iarc.fr/overtime/en/dataviz/trends?populations=752&sexes=1_2&types=1&multiple_populations=1.



In [ ]:
# Cargamos los datos
dt=pd.read_excel('DatosPractica_Scoring.xlsx', engine='openpyxl')

## Descripción inicial de los datos
Vamos a hacer una descripción inicial de los datos


In [ ]:
#Información del Contenido
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        1319 non-null   int64  
 1   Cardhldr  1285 non-null   float64
 2   default   994 non-null    float64
 3   Age       1319 non-null   float64
 4   Income    1319 non-null   float64
 5   Exp_Inc   1319 non-null   float64
 6   Avgexp    1319 non-null   float64
 7   Ownrent   1319 non-null   int64  
 8   Selfempl  1319 non-null   int64  
 9   Depndt    1319 non-null   int64  
 10  Inc_per   1319 non-null   float64
 11  Cur_add   1319 non-null   int64  
 12  Major     1319 non-null   int64  
 13  Active    1319 non-null   int64  
dtypes: float64(7), int64(7)
memory usage: 144.4 KB


## Variable objetivo: creditability

 La variable **creditability** es la *calidad crediticia* de  cada cliente, es la variable a predecir. Toma originalmente dos valores (Buen Cliente y Mal Clioente). Esta es la variable objetivo, la variable evento

In [ ]:
dt["Cardhldr"].value_counts()


Cardhldr
1.0    994
0.0    291
Name: count, dtype: int64

In [ ]:
#Recodifico esta variable creditability para que sea binaria y la llamo "y"

#dt.rename(columns={"creditability":"y"},inplace=True)
#dt['y'] = dt['y'].replace(['good', 'bad'], [0,1])
#Se construirá una tabla donde solamente hayan sido aceptado los clientes.
dt_aceptados = dt[dt['Cardhldr'] == 1]
display(dt_aceptados)

,ID,Cardhldr,default,Age,Income,Exp_Inc,Avgexp,Ownrent,Selfempl,Depndt,Inc_per,Cur_add,Major,Active
0,1,1.0,0.0,27.08333,2.4000,0.016798,33.01333,0,0,0,2.400000,56,1,1
1,2,1.0,1.0,24.25000,3.5000,0.069963,203.89170,0,0,0,3.500000,60,1,11
3,4,1.0,0.0,40.33333,3.0670,0.159700,408.08250,0,0,2,1.022333,18,0,0
4,5,1.0,0.0,28.16667,3.3500,0.071625,199.36920,1,0,0,3.350000,18,1,2
6,7,1.0,1.0,23.25000,1.8769,0.353630,553.10670,0,0,0,1.876900,12,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,1279,1.0,0.0,41.66667,2.4752,0.067250,138.54670,0,0,2,0.825067,222,0,10
1280,1281,1.0,0.0,40.66667,5.0500,0.028428,119.30170,1,0,0,5.050000,128,1,10
1282,1283,1.0,0.0,44.66667,2.4000,0.000500,0.00000,0,0,0,2.400000,36,1,11
1283,1284,1.0,0.0,47.41667,4.9200,0.044894,183.89830,1,1,1,2.460000,288,1,14


In [ ]:
#dt["y"].value_counts()
#ya filtrados los clientes, se calculan las VI
# Realizamos la trimificación optima de age.in.years

# Definir variable objetivo e independiente

# Elige variable y prepara datos
#variable = "Age"
#variable = "Income"
#variable = "Exp_Inc"
#variable = "Avgexp"
#variable = "Ownrent"
#variable = "Selfempl"
#variable = "Depndt"
#variable = "Inc_per"
#variable = "Cur_add"
#variable = "Major"
variable = "Active"


#dt_modelo = dt_aceptados.dropna(subset=["default", variable])

X = dt_aceptados[variable].values
Y = dt_aceptados["default"].values

# Crear y ajustar el binning
optb = OptimalBinning(name=variable, dtype="numerical")
optb.fit(X, Y)

# Mostrar los splits
print("Cortes óptimos:", optb.splits)

# Tabla de binning
binning_table = optb.binning_table
binning_table.build()

Cortes óptimos: [ 2.5  5.5  6.5  9.5 11.5 18.5]


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 2.50)",259,0.260563,256,3,0.011583,2.299735,0.595158,0.061393
1,"[2.50, 5.50)",203,0.204225,184,19,0.093596,0.123666,0.002974,0.000372
2,"[5.50, 6.50)",58,0.058350,52,6,0.103448,0.012654,0.000009,0.000001
3,"[6.50, 9.50)",168,0.169014,149,19,0.113095,-0.087323,0.001334,0.000167
4,"[9.50, 11.50)",83,0.083501,72,11,0.132530,-0.26806,0.006667,0.000831
5,"[11.50, 18.50)",168,0.169014,135,33,0.196429,-0.738063,0.122240,0.014942
6,"[18.50, inf)",55,0.055332,42,13,0.236364,-0.97411,0.075795,0.009117
7,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
8,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,994,1.000000,890,104,0.104628,,0.804177,0.086823


In [ ]:
dt.rename(columns={"default":"y"},inplace=True)
display(dt["y"].value_counts())
dt['y'].value_counts(normalize=True)

y
0.0    890
1.0    104
Name: count, dtype: int64

y
0.0    0.895372
1.0    0.104628
Name: proportion, dtype: float64

**Fíjate que tenemos un 30% de malos clientes**

In [ ]:
# Guardo los valores para su uso posterior
yT_0 = dt['y'].value_counts(normalize=True)[0]
yT_1 = dt['y'].value_counts(normalize=True)[1]
print(yT_0, yT_1)

0.8953722334004024 0.10462776659959759


In [ ]:
# También elimino todos los nan
dt.dropna(inplace=True)

dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 994 entries, 0 to 1284
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        994 non-null    int64  
 1   Cardhldr  994 non-null    float64
 2   y         994 non-null    float64
 3   Age       994 non-null    float64
 4   Income    994 non-null    float64
 5   Exp_Inc   994 non-null    float64
 6   Avgexp    994 non-null    float64
 7   Ownrent   994 non-null    int64  
 8   Selfempl  994 non-null    int64  
 9   Depndt    994 non-null    int64  
 10  Inc_per   994 non-null    float64
 11  Cur_add   994 non-null    int64  
 12  Major     994 non-null    int64  
 13  Active    994 non-null    int64  
dtypes: float64(7), int64(7)
memory usage: 116.5 KB


## Variables Predictoras o explicativas

Las 20 restantes variables del data frame (7 numéricas y 13 categóricas) son los atributos o características observadas de esos clientes que se utilizarán para predecir la probabilidad de que los clientes cometan un impago de sis créditos, esto es, de que sean malos clientes. La descripción de estas 20 variables es la siguiente:

In [ ]:
#  - Attribute 1: (qualitative) **Status of existing checking account o cuenta corriente**
#   - A11 : ... < 0 DM
#   - A12 : 0 <= ... < 200 DM
#   - A13 : ... >= 200 DM / salary assignments for at least 1 year
#   - A14 : no checking account

#- Attribute 2: (numerical) **Duration in month**

#  - Attribute 3: (qualitative) **Credit history**
#     - A30 : no credits taken/ all credits paid back duly (devultos sin mora)
#     - A31 : all credits at this bank paid back duly
#     - A32 : existing credits paid back duly till now
#     - A33 : delay in paying off in the past
#     - A34 : critical account/ other credits existing (not at this bank)

# - Attribute 4: (qualitative) **Purpose**
#     - A40 : car (new)
#     - A41 : car (used)
#     - A42 : furniture/equipment
#     - A43 : radio/television
#     - A44 : domestic appliances
#     - A45 : repairs
#     - A46 : education
#     - A47 : (vacation - does not exist?)
#     - A48 : retraining
#     - A49 : business
#    - A410 : others

# - Attribute 5: (numerical) **Credit amount**

# - Attribute 6: (qualitative) **Savings account/bonds**
#    - A61 : ... < 100 DM
#    - A62 : 100 <= ... < 500 DM
#    - A63 : 500 <= ... < 1000 DM
#    - A64 : .. >= 1000 DM
#    - A65 : unknown/ no savings account

# - Attribute 7: (qualitative) **Present employment since**
#     - A71 : unemployed
#     - A72 : ... < 1 year
#     - A73 : 1 <= ... < 4 years
#     - A74 : 4 <= ... < 7 years
#     - A75 : .. >= 7 years

# - Attribute 8: (numerical) **Installment rate in percentage of disposable income**

#  - Attribute 9: (qualitative) **Personal status and sex**
#     - A91 : male : divorced/separated
#     - A92 : female : divorced/separated/married
#     - A93 : male : single
#     - A94 : male : married/widowed
#     - A95 : female : single

# - Attribute 10: (qualitative) **Other debtors / guarantors**
#    - A101 : none
#    - A102 : co-applicant
#    - A103 : guarantor

# - Attribute 11: (numerical) **Present residence since**

# - Attribute 12: (qualitative) **Property**
#     - A121 : real estate
#     - A122 : if not A121 : building society savings agreement/ life insurance
#     - A123 : if not A121/A122 : car or other, not in attribute 6
#     - A124 : unknown / no property

# - Attribute 13: (numerical) **Age in years**

# - Attribute 14: (qualitative) **Other installment plans** Otros pagos por plazos
#      - A141 : bank
#      - A142 : stores
#      - A143 : none

# - Attribute 15: (qualitative) **Housing**
#      - A151 : rent
#      - A152 : own
#      - A153 : for free

# - Attribute 16: (numerical) **Number of existing credits at this bank**

# - Attribute 17: (qualitative) **Job**
#   - A171 : unemployed/ unskilled - non-resident
#   - A172 : unskilled - resident
#   - A173 : skilled employee / official
#   - A174 : management/ self-employed/highly qualified employee/ officer

# - Attribute 18: (numerical) **Number of people being liable to provide maintenance for**

# - Attribute 19: (qualitative) **Telephone**
#   - A191 : none
#   - A192 : yes, registered under the customers name

# - Attribute 20: (qualitative) **foreign worker**
#   - A201 : yes
#   - A202 : no



## Analisis univariante y de asociación con la variable objetivo

Que comprobar en el análisis descriptivo inicial?

### **Análisis univariante**

1- **Tipos de variables**. Todas las variables categóricas (factores) están bien identificadas?


Para el análisis univariante es importante realizar un análisis gráfico con el **Histograma de Frecuencias**

2- **Límites de las variables cuantitativas** Valores numéricos fuera de rango ¿Hay alguna limitación sobre el rango de alguna variable que no se cumpla?

3- **Niveles de las variables cualitativas**, Valores mal codificados ¿los niveles de las variables cualitativas tienen sentido? ¿Hay missings no declarados tipo -1, 99999? Las categorías de las nominales son las que deben?

4- **Variables nominales o categóricas o factores con categorías minoritarias**.Frecuencia de las categorías de las variables cualitativas.  Las categorías con baja representación puede causar muchos problemas en los modelos por falta de base muestral para la estimación de los parámetros correspondientes a la pertenencia a esa categoría. Por ello, es conveniente echar un vistazo y recodificar las vairables uniendo categorías muy poco representativas con otras cuya unión tenga algún sentido (tienen comportamiento similar frente a la objetivo, la variable tiene caracter ordinal por lo que la unión con mayor sentido sería hacia categorías adyacentes..).  Es imprescindible que todas los niveles de las variables cualitativas esten ´ bien representados pues, de lo contrario, se podr´ıan detectar patrones que no fueran extrapolables al estar basados en muy pocas observaciones.
 Por ello, se debe verificar que la frecuencia de todas ellas sea superior al 2-5 % (el porcentaje exacto depende del numero de observaciones del conjunto de datos) Yo prefiero que haya variabilidad suficiente: NO debe haber categorías con menos de un 5% de representación).

5 **Variabilidad suficiente de las variables numéricas** Por encima de un 5% de valores distintos


Con estas cosas ya arregladas, nos vamos a los dos grandes "caballos de batalla" de la depuración.

5- **Outliers**. Incidencia y tratamiento (pasar a missing, eliminar, winsorizar o reemplazarlos con los valores no atípicos más cercanos)

**Análisis de las variables/casos ausentes**


6- **Missings**. Incidencia y tratamiento
  - **Imputación** por valores validos (0-5%):  simple por media, mediana, aleatorio, imputación por modelos
  - **Recategorizacion** (5%- 50%) de los valores missing como una categoría valida.
  - **Eliminar** columnas u observaciones (superior al 50 %) Cuando en una variable hay mas de la mitad de los datos faltantes, es recomendable rechazarla al inicio del proceso, pues carece de suficiente informacion.
  

### **Análisis Bivariante**
**Después del análisis Univariante se realiza el Análisis Bivariante, entre cada una de las potenciales variables explicativas y la Variable objetivo**

a. Tablas de contigencia Chi2 (¿discretizar variables continuas V de Cramer o Chi2 dnormalizado entre 0 y 1,, independientes y totalmente dependientes respectivamente))
b. Tablas de correlaciones
c. Tablas Pivote (test de diferencia de medias)

c) Métodos gráficos:
  - Gráficos de dispersión
  - Diagrama de Cajas o boxplot (o diagrama de barras)
  - diagrams de mosaicos



***


## Análisis exploratorio inicial de las **variables categóricas**

Comenzamos con el análisis de las variables discretas

In [ ]:
dt.describe(include='object')

ValueError: No objects to concatenate

### **status.of.existing.checking.account**

In [ ]:
# tabla de distribución de frecuencia univariante
dt['status.of.existing.checking.account'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
# tabla de contingencia: distribución de frecuencias bi-variante
ctabla=pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
# test Chi-cuadrado de independencia (Ho: ausencia de asociación)
c, p, dof, expected = chi2_contingency(ctabla)
# Print the p-value
print(p)

In [ ]:
# tabla de contingencia agrupada por columnas (axis=1), Se puede analizar la importancia relativa comparando con los porcentajes clobales.
pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True, normalize=1).round(3)

In [ ]:
# Porcentajes respecto al total de cada fila (normalize=0), habría que comparar con el porcentaje de eventos en el total de la muestra (30%)
pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
pd.crosstab(dt['status.of.existing.checking.account'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))


In [ ]:
# Para comparar respecto a los totales

pd.crosstab(dt['status.of.existing.checking.account'], dt['y'], margins=True, normalize=0).round(3).plot(figsize=(15, 5))
plt.axhline(y=yT_0, color='#1f77b4', linestyle='--')
plt.axhline(y=yT_1, color='#ff7f0e', linestyle='--')
plt.show()

### **Personal status and sex**

In [ ]:
dt['personal.status.and.sex'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
ctabla=pd.crosstab(dt['personal.status.and.sex'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
# test Chi-cuadrado de independencia (Ho: ausencia de asociación)
c, p, dof, expected = chi2_contingency(ctabla)
# Print the p-value
print(p)

In [ ]:
pd.crosstab(dt['personal.status.and.sex'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
pd.crosstab(dt['personal.status.and.sex'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))
plt.axhline(y=yT_0, color='#1f77b4', linestyle='--')
plt.axhline(y=yT_1, color='#ff7f0e', linestyle='--')
plt.show()

### **Housing**

In [ ]:
dt['housing'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
ctabla=pd.crosstab(dt['housing'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
# test Chi-cuadrado de independencia (Ho: ausencia de asociación)
c, p, dof, expected = chi2_contingency(ctabla)
# Print the p-value
print(p)

In [ ]:
pd.crosstab(dt['housing'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
pd.crosstab(dt['housing'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))
plt.axhline(y=yT_0, color='#1f77b4', linestyle='--')
plt.axhline(y=yT_1, color='#ff7f0e', linestyle='--')
plt.show()

### **Job**

In [ ]:
dt['job'].value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
ctabla=pd.crosstab(dt['job'],dt['y'],margins=True).round(3)
ctabla

In [ ]:
# test Chi-cuadrado de independencia (Ho: ausencia de asociación)
c, p, dof, expected = chi2_contingency(ctabla)
# Print the p-value
print(p)

In [ ]:
pd.crosstab(dt['job'],dt['y'],margins=True, normalize=0).round(3)

In [ ]:
pd.crosstab(dt['job'],dt['y'],margins=True, normalize=0).round(3).plot(figsize=(15,5))
plt.axhline(y=yT_0, color='#1f77b4', linestyle='--')
plt.axhline(y=yT_1, color='#ff7f0e', linestyle='--')
plt.show()

 ....  Habría que seguir haciendo esto con todas las variables categóricas para analizar asociación

## Analisis univariante y de asociación con la variable objetivo de las **Variables continuas**

In [ ]:
dt.describe()

### **Credit.amount**

In [ ]:
sns.displot(x=(dt['credit.amount']), kind="kde", fill=True)
plt.show()

Parece que como la mayoría de variables económicas tipo cantidad de dinero (precios, salarios, rentas, etc) se comportan como una log normal. Lo comprobamos gráficamente

In [ ]:
sns.displot(x=np.log(dt['credit.amount']), kind="kde", fill=True)
plt.show()

Vamos a transformar la variable 'credit.amount' en logaritmos para conseguir normalidad (o al menos que se parezca a una normal)

In [ ]:
dt['credit.amount']=np.log(dt['credit.amount'])

Ahora vamos a ver si hay diferencias en la distribución de la variable 'credit.amount' entre los buenos y los malos clientes

In [ ]:
sns.displot(x=(dt['credit.amount']), kind="kde", fill=True, hue=dt.y, common_norm=False)
plt.show()

Según el histograma sí parece haber asociación, al menos hay tres tramos bien diferenciados, uno primero donde es difícil diferenciar entre los buenos y malos, otro tramo donde hay una mayor cantidad relativa de buenos clientes, y un tercero, en el que a partir de una determinada cantidad de crédito hay una mayoría relativa de malos clientes.    

Parece por tanto que sí hay asociación entre creditability (impago) y la cantidad de dinero que se solicita en el préstamo. PAra comprobarlo hacemos el test de diferencias de medias (Ho:igualdad de medias, esto es, ausencia de asociación)

In [ ]:
fvalue, pvalue = f_oneway(dt.loc[dt["y"]==0,['credit.amount']], dt.loc[dt["y"]==1,['credit.amount']])
print(fvalue, pvalue)


Hacemos la misma comprobación con el resto de variables

### **duration.in.month**

In [ ]:
sns.displot(x=(dt['duration.in.month']), kind="kde", fill=True)
plt.show()

In [ ]:
sns.displot(x=(dt['duration.in.month']), kind="kde", fill=True, hue=dt.y, common_norm=False)
plt.show()

In [ ]:
fvalue, pvalue = f_oneway(dt.loc[dt["y"]==0,['duration.in.month']], dt.loc[dt["y"]==1,['duration.in.month']])
print(fvalue, pvalue)

Tendría que continuar con el análisis del resto de variables continuas .....

In [ ]:
sns.pairplot(dt, hue="y")

# **Selección de Variables**: análisis de Concentración para seleccionar las variables más **importantes** para meter en el modelo

### Dividimos la muestra en entrenamiento y test     

Comenzamos ya el proceso de construcción del modelo en sentido estricto. Por eso lo primero es partir la muestra para comprobar la bondad del modelo que estimemos.



In [ ]:
dt_train, dt_test = train_test_split(dt,stratify=dt["y"], test_size=.25, random_state=1234)

In [ ]:
# Realizamos la trimificación optima de age.in.years
variable="age.in.years"
X=dt_train[variable].values
Y=dt_train['y'].values
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")
optb.fit(X, Y)
optb.splits
binning_table = optb.binning_table
binning_table.build()

In [ ]:
dt_train["y"].shape

In [ ]:
dt_train["y"].mean()

In [ ]:
dt_test["y"].shape

In [ ]:
dt_test["y"].mean()

## Defino la tramificación óptima

## Tramificación de la Variable: "credit.amount"

In [ ]:
variable="credit.amount"
X=dt_train[variable].values
Y=dt_train['y'].values

In [ ]:
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

# Si se quisiese fijar los intervalos manualmente (porque no gusten los que obtine el agoritmo, entonces habría que usar:
#                     user_splits=
#                     user_splits_fixed=
# HAy veces que los datos tienen dátos missing y códigos especiales en este caso para obtener una categoría con esos datos missing y datos especiales hay que establecerlos
#                     special_codes = [-9, -8, -7]

# Una vez definido podemos pasar a estimarlo
optb.fit(X, Y)
optb.splits

Nota: Por defecto se utiliza un arbol de clasificación para hacer una tramificación inicial, y después se aplica un proceso de optimización de agrupación de categorías para maximizar el Valor de Información

Una vez realizado el proceso de tramificación y agrupación óptima de categorías, obtenemos la tabla de agrupación

In [ ]:
binning_table = optb.binning_table
binning_table.build()

Cabe mencionar que el WOE en esta tabla parece estar definido al revés que lo hemos hecho en clase, por lo que el signo es justo el contrario al que cabría esperar según lo que hemos visto en clase. En particular por defecto `optbinning` define el WOE de una categoría $i$ como
$$ WOE_i =  ln \left ( {Non-event_i \over Non-event_{total}} \over {Event_i \over Event_{total}}    \right ) $$

En este sentido los niveles con mayor tasa de impagados tendrán un WOE menos, y a medida que se reduzca la tasa de impagados (mejor calidad crediticia) irá aumentando el WOE. De hecho, 'optbinning' ni siquiera utiliza la misma fórmula que yo he utilizado en clase, por lo que no está acotada entre cero y uno, puede valos más que uno sin que eso signifique sobre ajuste.

Por talmotivo utilizaremos como criterio de selección exclusivamente IV<0.002

Podemos extraer el IV y el índice de Gini a partir de la tabla

In [ ]:
print("IV= ", binning_table.iv.round(3))
print("Gini= ", binning_table.gini.round(3))

# La última columna muestra el estadístico Jensen-Shannon de divergencia.
# Es una medida de la similaridad entre dos distribuciones de probabilidad (frecuencias de buenos y malos )
# que está acotada entre 0 y log2 (aprox 0.70) (puede utilizarse 0.01 como mínimo)
print("JS= ", binning_table.js.round(3))



In [ ]:
# Podemos profundizar en el análisis estimando otras
binning_table.analysis(pvalue_test="chi2")


In [ ]:
# Por ejemplo otra medida que suele utilizarse en el Quality score(QS) que está acotada entre 0 y 1 (puede utilizarse 0.01 como mínimo)
print("QS= ", binning_table.quality_score.round(3))

In [ ]:
# La tabla anterior también muestra la V de Cramer (por encima de 0.20 podría ser suficiente para decir que hay asociación, entre los tramos y el porcentaje de eventos

# Pero también podemos realizar el test con la tabla de contigencia completa:Jensen-Sha
x_transform_indices = optb.transform(X, metric="indices")

#pd.Series(x_transform_indices).value_counts(normalize=True,dropna=False).sort_index()
ctabla=pd.crosstab(pd.Series(x_transform_indices),Y,margins=True).round(3)
print(ctabla)

# Chi-square test of independence. Ho: Ausencia de Asociación (independencia)
c, p, dof, expected = chi2_contingency(ctabla)
# Print the p-value
print("Test independencia. Estadístico :" ,round(c,3), "p-valor:", round(p,3))

Podemos realizar una representación gráfica de la Tabla de tramificación

In [ ]:
binning_table.plot(metric="woe")

In [ ]:
binning_table.plot(metric="event_rate")

Nótese que la relación entre lavariable x (credit.amount) y la tasa de evento (impago) es **no-lineal**

Ahora podemos aplicar esta tramificación óptima a la variable original y obtener la variable transformada WOE (que será una variable continua que utilizaremos en el modelode regresión)

In [ ]:
# Transformación WOE
x_woe = optb.transform(X, metric="woe")
pd.Series(x_woe).value_counts().sort_index()

Fíjate que ahora hemos conseguido "linealizar" la relación entre la variable trasnformada Woe y la propensión al impago

In [ ]:
pd.crosstab(x_woe,Y,normalize=0).round(3)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pd.crosstab(x_woe,Y,normalize=0).iloc[:,1])
ax.set_xlabel("x_woe")
ax.set_ylabel("porcentaje de impago")

Nótese que para hacer la validación deberíamos hacer exactamente la misma transformación WOE, con la misma tramificación, al conjunto test. Para ello debemos aplicar la transformación optima calculada con el conjunto de entrenamiento, pero sobre la muestra de validación

In [ ]:
# Transformación WOE en el conjunto test
x_test_woe = optb.transform(dt_test[variable].values, metric="woe")
pd.Series(x_test_woe).value_counts().sort_index()

Nótese que **no** estamos calculando una nueva tramificación para el conjunto de test, sino aplicando la tramificación obtenida con el conjunto de entrenamiento.    
En realidad si hiciéramos una tramificación óptima con el conjunto de test no tendría porqué salir igual que la estimada para el conunto de entrenamiento, como se puede comprobar a continuación

In [ ]:
variable="credit.amount"
X_test=dt_test[variable].values
Y_test=dt_test['y'].values
optb_test = OptimalBinning(name=variable, dtype="numerical")
optb_test.fit(X_test, Y_test)
print(optb_test.splits)
binning_table_test = optb_test.binning_table
binning_table_test.build()

Nótese que con el conjunto de test se han obtenido sólo 6 tramos y con diferentes puntos de corte ( y diferentes WOE), por eso es necesario no hacer una nueva tramificación al conjnto de test sino aplicar la tramificación obtenida usando en el conjunto de entrenamiento

## Tramificación de la duración en meses

In [ ]:
variable="duration.in.month"
X=dt_train[variable].values
Y=dt_train['y'].values

optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

optb.fit(X, Y)
optb.splits

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
print("IV= ", binning_table.iv.round(3))
print("Gini= ", binning_table.gini.round(3))


In [ ]:
binning_table.plot(metric="woe")

In [ ]:
binning_table.plot(metric="event_rate")

In [ ]:
# Transformación WOE
x_woe = optb.transform(X, metric="woe")
pd.Series(x_woe).value_counts().sort_index

Habría que seguir con el resto de variables continuas.....

# Agrupción de niveles en variables Vbles Categóricas

En realidad, cuando tenemosvariables categóricas, no es necesario tramificar, pero sí hacer una agrupación de los diferentes niveles de forma que se maximice el *valor de información*

## Agrupación de la variable  *purpose*

In [ ]:
variable_cat = "purpose"
X_cat = dt_train[variable_cat].values
Y_cat = dt_train['y'].values

dt_train[variable_cat].value_counts()

In [ ]:

optb = OptimalBinning(name=variable_cat, dtype="categorical", solver="cp",
                      cat_cutoff=0.1)  # podemos cambiar los valores por defecto cat_cutoff=None, o, cat_cutoff=0.005

optb.fit(X_cat, Y_cat)
optb.splits

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
binning_table.plot(metric="event_rate")

In [ ]:
x_woe = optb.transform(X_cat, metric="woe")
pd.Series(x_woe).value_counts()


... habría que seguir con el resto de variables categóricas

__________________________________________________________________________________________________________________________________________________________________________________________________
__________________________________________________________________________________________________________________________________________________________________________________________________

__________________________________________________________________________________________________________________________________________________________________________________________________
__________________________________________________________________________________________________________________________________________________________________________________________________

# Proceso de tramificación, agrupación y trasformación WOE Completo

Para no ir variable a variable se puede hacer todo el proceso completo

Proceso Entero



In [ ]:
# 1) Definimos la lista de nombres señalando cualse de ellas son las categóricas
Y = dt_train['y'].values
X = dt_train.drop(columns=['y']) #todas menos la primera que es el ID y la variable y
list_variables = X.columns.values.tolist()
list_categorical = X.select_dtypes(include=['object', 'category']).columns.values.tolist()

In [ ]:
# 2) Definimos el criterio de selección
selection_criteria = {
    "iv": {"min": 0.02}  # no imponemos "max": 1}
}

In [ ]:
# En caso de que desee modificarse los valores por defecto en el proceso de tramificación de alguna variable puede hacerse en forma de diccionario

binning_fit_params={
    "purpose":{"cat_cutoff": 0.10}
}

In [ ]:
# 3) Definimos el proceso de Tramificación o BinningProcess
binning_process = BinningProcess(
    categorical_variables=list_categorical,
    variable_names=list_variables,
    selection_criteria=selection_criteria,
    binning_fit_params=binning_fit_params)

In [ ]:
# 4) Obtenemos los tramos optimos de todas las Variables
dt_train_binned = binning_process.fit(X, Y)

In [ ]:
dt_train_binned.summary().sort_values('iv')

In [ ]:
# Ahora podemos ir sacando las tablas para cada variable

dt_train_binned.get_binned_variable("credit.amount").binning_table.build()

In [ ]:
dt_train_binned.get_binned_variable("purpose").binning_table.build()

In [ ]:
dt_train_binned.information()

In [ ]:
# las variables seleccionadas se pueden obtener con 'get_support'"Tarea Estudiantes_TarjetaPuntuacion"
dt_train_binned.get_support()

In [ ]:
# Podemos transformar las variables WOE
dt_train_woe=dt_train_binned.transform(X, metric="woe")


# Existe la posibilidad de obtener directamente las transformada si en lugar de usar fit, hubiésemos usado fit_transform
# dt_train_binned = binning_process.fit_transform(X, Y)
# dt_train_binned.info()
# el resultado sería un data.frame con las X seleccionadas trasnsformadas WOE



In [ ]:
dt_train_woe.info()

In [ ]:
dt_train_woe

In [ ]:
# Ahora aplicaríamos la misma transformación pero al conjunto de test (si hubiera que puntuar a nuevos clientes haríamos lo mismo)

Y_test = dt_test['y'].values
X_test = dt_test.drop(columns=['y']) #todas menos la primera que es el ID y la variable y

dt_test_woe=dt_train_binned.transform(X_test, metric="woe")
dt_test_woe.info()





In [ ]:
dt_test_woe

# Estimación del Modelo

Ahora podemos calcular la tarjeta de puntuación. En los apuntes de clase definimos tanto los WOE, como los Odd ratio como la probabilidad de `evento` respecto al `no-evento` (malos clientes o impago=1 respecto a los buenos clientes o impago=0):

$$ odd = {{P}\over {(1-P)}} ~~ {,~~  siendo} ~~  {P=Prob(impago=1)} $$

 Y la fórmula para obtener la puntuación o los score debe ser una relación negativa con los odd ratio: cuanto mayor la probabilidad de impago (en relación a la de no impago), menor puntuación ha de tener:

 $$ score {= offset - Factor}~·~{ln(odds)}$$

Para pasar de Probabiliddes de impago a Puntuaciones, habrá que establecer tanto el valor de `offset` como el de `Factor`. Esto se hace de manera arbitraria dependiendo de cada institución financiera.

En general, para determinar estos dos valores es necesario establecer la pendiente de la recta y un punto de la misma.

En cuanto a la pendiente, cuanto más plana sea la pendiente, menor variabilidad tendrán los valores de puntuación de crédito que se alcancen, y al revés, cuanto mayor pendiente más diferencias en la puntuación final. Yo voy a utilizar un apendiente (arbitraria) estableciendo de forma arbitraria cada cuantos puntos de score (**pdo_0**) se dobla el odd ratio: $ score - pdo_0 = {offset -Factor}~ ·{ln(2*odds)}$.

 En cuanto al punto de la recta (arbitrario), puede hacerse estableciento (de manera arbitraria) la puntuación o score considerada como de sobresaliente(**scorecard_points**) y el odd ratio que debería tener ese cliente de *sobresaliente* (**odds_0**)

 Así habría que establecer tres parámetros para transformar probabilidades de impago a puntuaciones, por ejemplo:   

* **pdo_0** =40  (esto es que cada 40 puntos de calidad creditica se dobla el odd-ratio))
* **scorecard_points** =600  (alguien con calidad crediticia muy buena, de sobresaliente, sacaría 600 puntos)
* **odds_0** =1/50  (odd ratio que se considera de sobresaliente)

La librería `optBinning` [librería OptBinning](http://gnpalencia.org/optbinning/), en realidad utiliza el módulo de `credit scoring` de `SAS-miner` como inspiración, y por eso define al revés tanto los WOE como los odd ratio, es decir `no-evento` en relación a `evento` (clientes buenos respecto a los malos, o no-impago respecto a impago, impago=0 respecto a impago=1).
$$ odd^B = {{(1-P)}\over {P}} ~~ {,~~  siendo} ~~  {P=Prob(impago=1)} $$

Esto implica que la ecuación que transforma las probabilidades de impago en scores utilizando esta *odds<sup>B</sup>* debe tener pendiente positiva (cuanto mejor *odd<sup>B</sup>* mejor calidad crediticia tiene el cliente)

 $$ score= {offset + Factor} ~·~ {ln(odds^B)}$$

 Nótese que ahora habrá que establecer de nuevo los puntos de score que doblan el odd ratio (**pdo_0**), y también la puntuación o score considerada como de sobresaliente(**scorecard_points**) y el odd ratio que debería tener ese cliente de *sobresaliente* **odds_0 <sup>*B*</sup>**, con **odds_0 <sup>*B*</sup>** **= 1/odds_0**.

 Así para estimar la puntuación crediticia con `optBinning` hay que establecer tres parámetros para transformar probabilidades de impago a puntuaciones, por ejemplo:   

* **pdo_0** =40
* **scorecard_points** =600
* **odds_0 <sup>*B*</sup>** = 50  (equivalente a **odds_0** =1/50 )




In [ ]:

# Directamente con el método Scorecard
estimator = LogisticRegression(solver="lbfgs")

# Establecemos los parámetros para la transformación de probabilidades en puntos de calidad crediticia o score

pdo_0 =40
scorecard_points_0= 600
odds_0_B= 50 # (equivalente a  odds_0 =1/50 )

tarjeta= Scorecard(binning_process=binning_process,
                   estimator=estimator,
                   scaling_method="pdo_odds",
                   scaling_method_params={"pdo":pdo_0, "odds": odds_0_B, "scorecard_points": scorecard_points_0})

tarjeta.fit(X, Y, show_digits=4)

In [ ]:
# Podemos obtener los parámetros del modelo de regresión logística
tarjeta.table(style="detailed")[["Variable","Coefficient"]].groupby(["Variable", "Coefficient"]).nunique()

Y podemos obtener los puntos de la tarjeta de puntuación

In [ ]:
tarjeta.table().head(60)

In [ ]:
#Aquí se obtienen todos los estadísticos
tarjeta.table(style="detailed").head(16)

Continuamos con la Diagnosis del Modelo

In [ ]:
# obtenmos las predicciones
Y_pred=tarjeta.predict_proba(X)[:,1]

# Calculamos la media
Y_pred.mean().round(4)


Para contruir las matrices de confusión necesitamos determinar un **punto de corte de la probabilidad**.    

Ese punto de corte es el que me va a ayudar a realizar un pronóstico sobre los clientes: los malos clientes serán aquellos para los que Prob Estimada > Prob_corte.


In [ ]:
# Para elegir el punto de corte puede utilizarse el Plot Kolmogorov-Smirnov (KS)
plot_ks(Y, Y_pred)

In [ ]:
# También puede utilizarse el máximo del f1_score
# definimos un vector de puntos de corte
c=np.arange(0,1,0.01)
# calculamos el f1_score para cada punto de corte
f1_score_ = [f1_score(dt_train["y"],np.multiply(Y_pred>c_,1)) for c_ in c]
# obtenemos el punto de corte que maximiza el f1_score
c_max = c[np.argmax(f1_score_)]
print("El punto de corte que maximiza el f1_score es: ", c_max)
print("y el máximo se alcanza en ", np.max(f1_score_).round(3))

# hacemos un gráfico de c y los f1_score correspondientes
plt.plot(c,f1_score_)
plt.stem(c_max, np.max(f1_score_),linefmt='r--', markerfmt='ro', basefmt='r--')
plt.title("Gráfico f1_score vs diferentes puntos de corte")
plt.text(c_max+0.05, 0, "c_max = "+str(c_max.round(2))+"\n f1_score_max = "+str(np.max(f1_score_).round(3)))
plt.show()

c_maxF1=c_max

In [ ]:
# método de Youden (J) para obtener el punto de corte óptimo
# definimos un vector de puntos de corte (c) y de probabilidad de aceptación (p)
c=np.arange(0,1,0.01)

# Calculamos el estadístico J de Youden para cada punto de corte= Sensibilidad + Especificidad -1
J= [balanced_accuracy_score(dt_train["y"],np.multiply(Y_pred>c_,1), adjusted=True) for c_ in c ]
# obtenemos el punto de corte que maximiza el índice de Youden
c_max = c[np.argmax(J)]
print("El punto de corte que maximiza el índice de Youden es: ", c_max)
print("y el máximo se alcanza en ", np.max(J).round(3))

# gráfico de c y los índices de Youden correspondientes
plt.plot(c,J)
plt.stem(c_max, np.max(J),linefmt='r--', markerfmt='ro', basefmt='r--')
plt.title("Gráfico índice de Youden vs diferentes puntos de corte")
plt.text(c_max+0.05, 0, "c_max = "+str(c_max.round(2))+"\n J_max = "+str(np.max(J).round(3)))
plt.show()

In [ ]:
# Para el punto de corte se podría utilizar simplemente la frecuencia observada, o utilizar el punto donde se alcanza el maximo del F1 Score
Prob_Corte=Y.mean()
# Prob_Corte=c_maxF1
print(' Punto de Corte seleccionado:', Prob_Corte.round(2),'\n',
      'Frecuencia media de eventos (y=1):', Y.mean().round(2), '\n',
      'máximo del F1-score :', c_maxF1)

Ahora ya podemos hacer los pronósticos

In [ ]:
dt_train["Y_pronostico"]=np.multiply(Y_pred>Prob_Corte,1)

Para comprobar la bondad de nuestras predicciones voy a comparar resultados con la tabla de confusión

In [ ]:


# Primero estimo la precisión (los aciertos sobre el total de mis pronósticos)
pd.crosstab(dt_train["y"],dt_train["Y_pronostico"],margins=True, normalize=1).round(3)


Nótese que los falsos negativos (Bad Rate) es del 11.5% (préstamos aceptados o pronosticados como buenos que resultaron impagados)

In [ ]:
# Ahora estimo la exhaustividad o recall (Aciertos sobre los casos reales):
# la sensibilidad (sobre los positivos y=1), y la Especificidad (sobre los negativos y=0)

pd.crosstab(dt_train["y"],dt_train["Y_pronostico"],margins=True, normalize=0).round(3)

In [ ]:
# Por último un resumen global de aciertos
f1_score(dt_train["y"],dt_train["Y_pronostico"])

Para hacer la diagnosis también puedo utilizar medidas que no dependan crucialmente de un único punto de corte de Probabilidad

In [ ]:
# Diagnosis Curva ROC
plot_auc_roc(Y,Y_pred)

In [ ]:
# Diagnosis Cumulative Accuracy Profile (CAP)
# Otra curva alternativa a la curva ROC que permite evaluar la bondad de un modelo de clasificación es la curva CAP (Cumulative Accuracy Profile).
# La curva CAP se construye de la siguiente manera: ordenamos las observaciones de mayor a menor probabilidad
# de pertenecer a la clase positiva (y=1). A continuación, vamos acumulando las observaciones y calculando
# la proporción de positivos acumulados sobre el total de positivos. Esta proporción se representa en el eje Y.

# En el eje X representamos la proporción de observaciones acumuladas sobre el total de observaciones.
# La curva CAP se construye a partir de la curva de la línea recta (curva de referencia) y
# la curva de la línea que representa la probabilidad estimada por el modelo.

plot_cap(Y, Y_pred)

In [ ]:
#### OJO que la diagnosis debe hacerse fuera de la muestra de entrenamiento
# obtenmos las predicciones
Y_test_pred=tarjeta.predict_proba(X_test)[:,1]

# Calculamos la media
Y_test_pred.mean().round(5)

In [ ]:
# Diagnosis Curva ROC
plot_auc_roc(Y_test,Y_test_pred)


In [ ]:
plot_cap(Y_test,Y_test_pred)

In [ ]:
dt_test["Y_pronostico"]=np.multiply(Y_test_pred>Prob_Corte,1)

# Primero estimo la precisión (los aciertos sobre el total de mis pronósticos)
print("\n Precisión:\n", pd.crosstab(dt_test["y"],dt_test["Y_pronostico"],margins=True, normalize=1).round(3))

# Ahora estimo la exhaustividad o recall (Aciertos sobre los casos reales):
# la sensibilidad (sobre los positivos y=1), y la Especificidad (sobre los negativos y=0)
print("\n exhaustividad:\n",pd.crosstab(dt_test["y"],dt_test["Y_pronostico"],margins=True, normalize=0).round(3))

# Por último un resumen global de aciertos
print("\n f1-score:",f1_score(dt_test["y"],dt_test["Y_pronostico"]))

Pasamos ahora de Probabilidades a Puntuaciones

In [ ]:
# Ahora vamos a calcular los score o puntuaciones.
# Que podemos hacer  con la función score
score = tarjeta.score(X)

print("Puntuación mínima: ", score.min().round(2))
print("Puntuación máxima: ",score.max().round(2))
print("Puntuación media : ",score.mean().round(2))


# O también haber calculado la puntuación manualmente (voy a calcularla para comprobar que da exactamente el mismo resultado)

# Transformación lineal según apuntes
# Factor= (pdo_0/log(2))
# Offset = scorecard_points_0+(pdo_0/log(2))*log(odds0_0)
# score= Offset - Factor *log(odds)

Factor= (pdo_0/np.log(2))
Offset = scorecard_points_0+Factor*np.log(1/odds_0_B)

score2= Offset-Factor*np.log(Y_pred/(1-Y_pred))

# Podemos comprobar que los resultados son los mismos
print("Puntuación mínima cálculo manual: ",score2.min().round(2))
print("Puntuación máxima cálculo manual: ",score2.max().round(2))
print("Puntuación media cálculo manual : ",score2.mean().round(2))

datos_score=pd.DataFrame(np.transpose([score,score2, Y,Y_pred]), columns=['score','scoreManual','Y','Y_pred'])

# si es necesario podemos guardar los datos
# datos_score.to_excel("score_p1.xlsx")


Saber cómo se hace la transformación manual puede ayudarnos por ejemplo a la hora de establecer la `nota que determina el aprobado`. Imaginemos que utilizamos la frecuencia observada de impagos como probabilidad de corte

In [ ]:

Score_Corte= Offset-Factor*np.log(Prob_Corte/(1-Prob_Corte))

print("La probabilida de corte de: ", Prob_Corte, " equivale a una puntuación de corte de: ", Score_Corte.round(2) )


In [ ]:
# Ahora representamos en un gráfico cómo separa el modelo a los buenos y los malos
datos_score=pd.DataFrame(np.transpose([score,Y]), columns=['score','Y'])
sns.displot(data=datos_score, x='score', label="event", hue='Y', alpha=0.35,kind="kde", fill=True, common_norm=True)
plt.axvline(Score_Corte, color='k', linestyle=":")

# Seguimiento del modelo: PSI (Population Stability Index)
El PSI es una medida de diferencia en la distribución de dos muestras, en nuestro caso entre la muestra utilizada para construir el modelo (entrenar y validar el modelo), y los nuevos datos que se vayan obteniendo con el transcurso del tiempo.  

Se aplica para detectar cuándo comienzan a verse diferencias entre las dos muestras (las puntuaciones de la muestra -train- y las puntuaciones obtenidas con los nuevos datos .... Cuando las distribuciones dejen de parecerse será el momento de revisar el modelo a tenor de los nuevos datos

Como regla general
  - **PSI <0.1**: No hay diferencias significativas entre las muestras de entrenamiento y los nuevos datos (resultado deseado, no se requiere más acciones)
  - **PSI entre 0.1 y 0.25** Hay cambio menores, valdría la pena revisar el modelo
  - **PSI >0.25** hay cambios importantes entre las dos muestras HAY QUE CAMBIAR EL MODELO


In [ ]:
# Supongamos que tenemos un conjunto de nuevos datos que hemos ido recopilando después de la puesta en producción del modelo,
# y queremos utilizar esos nuevos datos para saber si es necesario revisar el modelo o si por el contrario podemos seguir utilizándolo

# Como en la base de datos no disponemos de este tipo de datos voy a suponer simplemente que los datos de test son los nuevos datos,

dt_nuevosdatos= dt_test.copy()


# Valores nuevos
Y_nuevo = dt_nuevosdatos['y'].values
X_nuevo = dt_nuevosdatos.drop(columns=['y']) #todas menos la primera que es el ID y la variable y

# Valores de entrenamiento
Y = dt_train['y'].values
X = dt_train.drop(columns=['y']) #todas menos la variable y




In [ ]:
# ¿se distibuyen igual las probabilidades esperadas?
score_train = tarjeta.score(X)
score_nuevo = tarjeta.score(X_nuevo)

datos_score_psi1=pd.DataFrame(np.transpose([score_train,Y]), columns=['score','Y'])
datos_score_psi1['tipo']='entrenamiento'

datos_score_psi2=pd.DataFrame(np.transpose([score_nuevo,Y_nuevo]), columns=['score','Y'])
datos_score_psi2['tipo']='nuevos datos'

datos_score_psi= pd.concat([datos_score_psi1,datos_score_psi2])
sns.displot(data=datos_score_psi, x='score', label="event", hue='tipo', alpha=0.35,kind="kde", fill=True,common_norm=False)
plt.axvline(Score_Corte, color='k', linestyle=":")
plt.show()

Aparentemente las distribuciones son parecidas, eso quiere decir que los nuevos datos se parecen a los utilizados en el entrenamiento del modelo, por lo que seguramente no es necesario revisar el modelo. Lo comprobamos de todas formas con el estadístico PSI (Population Stability Index)

In [ ]:
# Estimo el psi
# Defino la tarjeta a evalear
# psi=ScorecardMonitoring(tarjeta, psi_method="cart",psi_min_bin_size=0.05, psi_n_bins=13)
psi=ScorecardMonitoring(tarjeta, psi_method= "quantile", psi_n_bins=10)
psi.fit(X_actual=X_nuevo, y_actual=Y_nuevo, X_expected=X, y_expected=Y)

psi.psi_plot()
psi.psi_table()
psi.tests_table()


In [ ]:
psi.system_stability_report()

In [ ]:
psi.psi_variable_table(style="summary").sort_values('PSI')